In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model

base_dir = "/kaggle/input/image-super-resolution/dataset/"

## data normalization

In [ ]:
def process_image(image):
    return image/255

## loading the dataset and splitting it

In [ ]:
def load_data(path):
    high_res_images = []
    low_res_images = []
    for dirname, _, filenames in os.walk(path+'low_res'):
        for filename in filenames:
            img = cv2.imread(os.path.join(dirname, filename))
            img = process_image(img)
            low_res_images.append(img)
        
    for dirname, _, filenames in os.walk(path+'high_res'):
        for filename in filenames:
            img = cv2.imread(os.path.join(dirname, filename))
            img = process_image(img)
            high_res_images.append(img)
    
    return np.array(low_res_images), np.array(high_res_images)

In [ ]:
x_train, y_train =  load_data(base_dir+'train/')
x_test, y_test = load_data(base_dir+'val/')

## dataset size

In [ ]:
print(x_test[2].ndim)
print(x_test[2].shape)
print(x_test[18].size)

## plotting some images

In [ ]:
n = 5
plt.figure(figsize=(60, 10))
for i in range(n):
    ax = plt.subplot(1, n, i + 1)
    plt.title("original + noise")
    plt.imshow(x_test[i+18])
    plt.title(x_test[i+18].shape)
    plt.gray()
plt.show()

## model building

In [ ]:
input= layers.Input(shape=(256,256,3))
x = layers.Conv2D(256, (3,3), activation="relu", padding="same")(input)
x = layers.MaxPooling2D((2,2), padding="same")(x)
x = layers.Conv2D(128, (3,3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2,2), padding="same")(x)
x = layers.Conv2D(64, (3,3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2,2), padding="same")(x)
x = layers.Conv2D(32, (3,3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2,2), padding="same")(x)
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D(2, padding="same")(x)
# Decoder
x = layers.Conv2DTranspose(32, (3,3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(32, (3,3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(64, (3,3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(128, (3,3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2DTranspose(256, (3,3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(1, (3,3), activation="sigmoid", padding="same")(x)
# Autoencoder
autoencoder = Model(input, x)
autoencoder.compile(optimizer="adam", loss=losses.MeanSquaredError())
autoencoder.summary()

## model architecture

In [ ]:
autoencoder.summary()

## model trainning

In [ ]:
autoencoder.fit(x_train, y_train,
                epochs=10,
                shuffle=True,
                validation_data=(x_test ,y_test))

In [ ]:
autoencoder.predict(x_test[2])

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):

    # display original + noise
    ax = plt.subplot(2, n, i + 1)
    plt.title("original + noise")
    plt.imshow(x_test[i])
    plt.gray()

    # display reconstruction
    bx = plt.subplot(2, n, i + n + 1)
    plt.title("reconstructed")
    plt.imshow(autoencoder.predict(x_test[i]))
plt.show()